In [2]:
!pip install -q pandas numpy requests beautifulsoup4 matplotlib

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory


In [34]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
import os
import glob

In [16]:
def obter_dados_fundamentus():
    url = "https://www.fundamentus.com.br/resultado.php"
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0'
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    tabela = soup.find('table', {'id': 'resultado'})
    dados = []

    # Extrair cabeçalhos
    headers = []
    for th in tabela.find_all('th'):
        headers.append(th.text.strip())

    # Extrair linhas de dados
    for tr in tabela.find_all('tr')[1:]:
        row = {}
        for i, td in enumerate(tr.find_all('td')):
            row[headers[i]] = td.text.strip()
        dados.append(row)

    # Converter para DataFrame
    df = pd.DataFrame(dados)

    print(headers)

    # Limpar formatação dos dados numéricos, excluindo a primeira coluna (o nome da ação)
    for col in headers[1:]:
        if col in df.columns:
            df[col] = df[col].str.replace('.', '').str.replace(',', '.')
            df[col] = df[col].str.replace('%', '')
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

In [17]:
acoes_df = obter_dados_fundamentus()
acoes_df

['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a']


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,PORP4,2.40,0.00,0.00,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,-2.08,0.0,2.239900e+07,0.00,13.66
1,POPR4,10.17,0.00,0.00,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,8.66,5.65,1.08,15.25,19.93,0.0,5.458030e+08,0.82,30.93
2,MNSA3,0.42,0.00,0.00,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,-208.15,-362.66,3.63,-13.50,145.70,0.0,-9.105000e+06,-6.52,-41.11
3,CFLU4,1000.00,0.00,0.00,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,8.88,10.72,1.10,17.68,32.15,0.0,6.035100e+07,0.06,8.14
4,CSTB4,147.69,0.00,0.00,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,40.85,28.98,2.60,22.40,20.11,0.0,8.420670e+09,0.14,31.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,UBBR4,7.49,610.27,1.99,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+10,0.00,10.58
987,VSTE3,6.80,1165.72,0.74,0.697,0.70,0.468,4.66,18.51,-6.20,...,4.98,3.77,0.06,1.52,2.80,0.06,113078.0,1.043280e+09,0.33,15.88
988,UBBR11,14.75,1201.81,3.91,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+10,0.00,10.58
989,UBBR3,18.00,1466.61,4.77,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+10,0.00,10.58


In [19]:
def obter_dados_historicos(tickers, periodo='1y'):
    dados_historicos = {}

    for ticker in tickers:
        try:
            ticker_yf = f"{ticker}.SA"
            data = yf.download(ticker_yf, period=periodo, progress=False)
            if len(data) > 0:
                dados_historicos[ticker] = data
        except Exception as e:
            print(f"Erro ao obter dados de {ticker}: {e}")
        finally:
            time.sleep(0.1)  # Evitar sobrecarga da API

    return dados_historicos

In [20]:
acoes_historico = obter_dados_historicos(acoes_df['Papel'].tolist(), periodo='1y')

YF.download() has changed argument auto_adjust default to True



1 Failed download:
['PORP4.SA']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['POPR4.SA']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['MNSA3.SA']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['CFLU4.SA']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['CSTB4.SA']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['CSTB3.SA']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/IVTT3.SA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=IVTT3.SA&crumb=OYBYbJJZ.9H

1 Failed download:
['IVTT3.SA']: AttributeError("'NoneType' object h

In [28]:
acoes_historico

{'SEER3': Price         Close     High      Low     Open   Volume
 Ticker     SEER3.SA SEER3.SA SEER3.SA SEER3.SA SEER3.SA
 Date                                                   
 2024-04-11     6.71     6.95     6.67     6.88   364100
 2024-04-12     6.36     6.76     6.35     6.76   683200
 2024-04-15     5.40     6.37     5.39     6.35  2525800
 2024-04-16     5.60     5.62     5.12     5.47  2280300
 2024-04-17     5.35     5.78     5.26     5.72  1296000
 ...             ...      ...      ...      ...      ...
 2025-04-07     4.91     5.09     4.74     4.97  1305300
 2025-04-08     4.89     5.17     4.81     4.92   937900
 2025-04-09     5.30     5.46     4.82     4.83  1807300
 2025-04-10     5.20     5.38     5.11     5.30  1249200
 2025-04-11     5.27     5.37     5.14     5.25   719100
 
 [252 rows x 5 columns],
 'FRIO3': Price         Close     High      Low     Open   Volume
 Ticker     FRIO3.SA FRIO3.SA FRIO3.SA FRIO3.SA FRIO3.SA
 Date                                      

In [71]:
def tratar_csv_acao(dados, ticker):
    """
    Trata o formato do CSV de ações removendo linhas desnecessárias
    e ajustando o cabeçalho

    Args:
        dados: DataFrame com os dados originais ou caminho para o arquivo CSV
        ticker: Ticker da ação

    Returns:
        DataFrame tratado com o formato correto
    """
    # Verificar se dados é um DataFrame ou um caminho para arquivo
    if isinstance(dados, str):
        # Se for um caminho, carregar o CSV
        dados = pd.read_csv(dados)

    # Pular as duas primeiras linhas após o cabeçalho
    dados_tratados = dados.iloc[3:].copy()

    # Renomear as colunas, removendo o nome do ticker de cada coluna
    colunas_originais = dados.columns.tolist()
    colunas_novas = [nome_col for nome_col, _ in colunas_originais]
    dados_tratados.columns = colunas_novas

    return dados_tratados

In [72]:
def salvar_dados_historicos_csv(dados_historicos, diretorio='dados_acoes'):
    """
    Salva os dados históricos das ações em arquivos CSV

    Args:
        dados_historicos: Dicionário com ticker como chave e DataFrame como valor
        diretorio: Diretório onde os arquivos CSV serão salvos

    Returns:
        Lista de arquivos CSV salvos
    """

    if not os.path.exists(diretorio):
        os.makedirs(diretorio)

    arquivos_salvos = []

    for ticker, dados in dados_historicos.items():
        try:
            caminho_arquivo = os.path.join(diretorio, f"{ticker}.csv")

            dados_tratado = tratar_csv_acao(dados, ticker)

            dados_tratado.to_csv(caminho_arquivo)
            arquivos_salvos.append(caminho_arquivo)
            print(f"Dados de {ticker} salvos em {caminho_arquivo}")
        except Exception as e:
            print(f"Erro ao salvar {ticker}: {e}")

    print(f"\nTotal de {len(arquivos_salvos)} arquivos CSV salvos no diretório '{diretorio}'")
    return arquivos_salvos

In [73]:
arquivos = salvar_dados_historicos_csv(acoes_historico)

Dados de SEER3 salvos em dados_acoes/SEER3.csv
Dados de FRIO3 salvos em dados_acoes/FRIO3.csv
Dados de USIM6 salvos em dados_acoes/USIM6.csv
Dados de AMBP3 salvos em dados_acoes/AMBP3.csv
Dados de CLSA3 salvos em dados_acoes/CLSA3.csv
Dados de OPCT3 salvos em dados_acoes/OPCT3.csv
Dados de FICT3 salvos em dados_acoes/FICT3.csv
Dados de AURA33 salvos em dados_acoes/AURA33.csv
Dados de USIM5 salvos em dados_acoes/USIM5.csv
Dados de USIM3 salvos em dados_acoes/USIM3.csv
Dados de PETZ3 salvos em dados_acoes/PETZ3.csv
Dados de ALPK3 salvos em dados_acoes/ALPK3.csv
Dados de DTCY3 salvos em dados_acoes/DTCY3.csv
Dados de CASH3 salvos em dados_acoes/CASH3.csv
Dados de JALL3 salvos em dados_acoes/JALL3.csv
Dados de RAIL3 salvos em dados_acoes/RAIL3.csv
Dados de FIEI3 salvos em dados_acoes/FIEI3.csv
Dados de DOHL3 salvos em dados_acoes/DOHL3.csv
Dados de SCAR3 salvos em dados_acoes/SCAR3.csv
Dados de HAPV3 salvos em dados_acoes/HAPV3.csv
Dados de MAPT4 salvos em dados_acoes/MAPT4.csv
Dados de MA

In [75]:
def carregar_arquivos_csv(diretorio='dados_acoes'):
    """
    Carrega todos os arquivos CSV de ações de um diretório

    Args:
        diretorio: Caminho para o diretório contendo os arquivos CSV

    Returns:
        Um dicionário com o ticker como chave e o DataFrame como valor
    """
    arquivos = glob.glob(os.path.join(diretorio, "*.csv"))
    acoes = {}

    for arquivo in arquivos:
        ticker = os.path.basename(arquivo).split('.')[0]
        try:
            acoes[ticker] = pd.read_csv(arquivo)
            print(f"Arquivo {ticker}.csv carregado com sucesso")
        except Exception as e:
            print(f"Erro ao carregar {ticker}.csv: {e}")

    print(f"\nTotal de {len(acoes)} arquivos carregados")
    return acoes

In [76]:
acoes_historico_dict = carregar_arquivos_csv()
acoes_historico_dict

Arquivo DIRR3.csv carregado com sucesso
Arquivo OIBR4.csv carregado com sucesso
Arquivo BRKM3.csv carregado com sucesso
Arquivo PEAB3.csv carregado com sucesso
Arquivo ALPA4.csv carregado com sucesso
Arquivo DEXP4.csv carregado com sucesso
Arquivo MGEL3.csv carregado com sucesso
Arquivo DXCO3.csv carregado com sucesso
Arquivo GPIV33.csv carregado com sucesso
Arquivo BGIP3.csv carregado com sucesso
Arquivo AZZA3.csv carregado com sucesso
Arquivo SHUL4.csv carregado com sucesso
Arquivo EMAE4.csv carregado com sucesso
Arquivo BBAS3.csv carregado com sucesso
Arquivo IGTI11.csv carregado com sucesso
Arquivo LUPA3.csv carregado com sucesso
Arquivo FIGE4.csv carregado com sucesso
Arquivo SANB3.csv carregado com sucesso
Arquivo CTSA3.csv carregado com sucesso
Arquivo ABEV3.csv carregado com sucesso
Arquivo CPLE6.csv carregado com sucesso
Arquivo RDNI3.csv carregado com sucesso
Arquivo EKTR3.csv carregado com sucesso
Arquivo LOGG3.csv carregado com sucesso
Arquivo EQPA7.csv carregado com sucess

{'DIRR3':            Date      Close       High        Low       Open   Volume
 0    2024-04-16  19.909971  20.034131  19.475409  19.892234  3093300
 1    2024-04-17  19.768072  20.371134  19.732597  19.954312  1542400
 2    2024-04-18  19.972050  20.078472  19.697122  19.865626  1682200
 3    2024-04-19  19.750336  20.326793  19.750336  19.909970  2159200
 4    2024-04-22  19.918835  20.025258  19.528618  19.794675  1359100
 ..          ...        ...        ...        ...        ...      ...
 244  2025-04-07  32.599998  32.790001  31.360001  31.740000  1982100
 245  2025-04-08  32.799999  33.660000  32.470001  32.750000  2663900
 246  2025-04-09  32.849998  33.400002  31.930000  32.700001  2077400
 247  2025-04-10  32.540001  32.910000  32.130001  32.910000  2753300
 248  2025-04-11  32.299999  32.680000  31.150000  32.500000  2925000
 
 [249 rows x 6 columns],
 'OIBR4':            Date      Close       High        Low       Open  Volume
 0    2024-04-16  18.700001  19.100000  18.500

In [99]:
# https://www.dias-uteis.com/#am1d
dias_uteis_6meses = 122
dias_uteis_12meses = dias_uteis_6meses * 2


def calcular_metricas_tecnicas(dados_historicos):
    resultados = []

    for i, (ticker, data) in enumerate(dados_historicos.items()):
        if len(data) < 50:  # Ignorar ações com poucos dados
            continue

        try:
            # Calcular retornos diários
            retornos = data['Close'].pct_change().dropna()

            preco_atual = data.iloc[-1]['Close']

            # Momentum de 6 meses
            range_6m = max(0, dias_uteis_6meses)
            if len(data) >= range_6m:
                preco_6m = data.iloc[-range_6m]['Close']
                momentum_6m = ((preco_atual / preco_6m) - 1) * 100
            else:
                momentum_6m = None

            # Momentum de 12 meses
            range_12m = max(0, dias_uteis_12meses)
            if len(data) >= range_12m:
                preco_12m = data.iloc[-range_12m]['Close']
                momentum_12m = ((preco_atual / preco_12m) - 1) * 100
            else:
                momentum_12m = None

            # Volatilidade (anualizada)
            volatilidade = retornos.std() * np.sqrt(dias_uteis_12meses) * 100

            resultados.append({
                'Ticker': ticker,
                'Momentum_6m': momentum_6m,
                'Momentum_12m': momentum_12m,
                'Volatilidade': volatilidade
            })

        except Exception as e:
            print(f"Erro nos cálculos para {ticker}: {e}")

    resultados_df = pd.DataFrame(resultados).loc[~(pd.DataFrame(resultados) == 0).any(axis=1)]
    return resultados_df

In [100]:
acoes_resultados = calcular_metricas_tecnicas(acoes_historico_dict)
acoes_resultados

,Ticker,Momentum_6m,Momentum_12m,Volatilidade
0,DIRR3,14.061952,60.020649,30.598054
1,OIBR4,-38.640593,-63.591160,154.357686
2,BRKM3,-42.911877,-52.482916,41.755155
3,PEAB3,-9.011195,-9.735953,42.284061
4,ALPA4,5.050511,-21.720429,35.454403
...,...,...,...,...
461,TXRX4,-12.711863,-10.434783,57.896237
462,CEBR3,-1.331686,-29.705595,34.374534
463,VSTE3,-33.916422,-54.374358,52.341992
465,JFEN3,-54.423077,-92.500000,129.762323


In [101]:
def ranking_fatores(df, fatores):
    """
    Calcula ranking para cada fator
    fatores = [{
        'nome': nome do fator,
        'coluna': coluna no dataframe,
        'ascendente': True se menor valor é melhor
    }]
    """
    df_ranking = df.copy()

    for fator in fatores:
        nome = fator['nome']
        coluna = fator['coluna']
        ascendente = fator['ascendente']

        if coluna in df.columns:
            df_ranking.loc[:, f'{nome}_Rank'] = df[coluna].rank(ascending=ascendente, method='min', na_option='bottom')

    return df_ranking

In [109]:
acoes_fatores = [
    {
        'nome': 'Momentum_6m',
        'coluna': 'Momentum_6m',
        'ascendente': False
    },
    {
        'nome': 'Momentum_12m',
        'coluna': 'Momentum_12m',
        'ascendente': False
    },
    {
        'nome': 'Volatilidade',
        'coluna': 'Volatilidade',
        'ascendente': True
    }
]
acoes_rank_fatores = ranking_fatores(acoes_resultados, acoes_fatores)

acoes_rank_fatores

,Ticker,Momentum_6m,Momentum_12m,Volatilidade,Momentum_6m_Rank,Momentum_12m_Rank,Volatilidade_Rank
0,DIRR3,14.061952,60.020649,30.598054,66.0,15.0,144.0
1,OIBR4,-38.640593,-63.591160,154.357686,389.0,404.0,421.0
2,BRKM3,-42.911877,-52.482916,41.755155,404.0,390.0,241.0
3,PEAB3,-9.011195,-9.735953,42.284061,237.0,221.0,244.0
4,ALPA4,5.050511,-21.720429,35.454403,117.0,283.0,194.0
...,...,...,...,...,...,...,...
461,TXRX4,-12.711863,-10.434783,57.896237,278.0,228.0,354.0
462,CEBR3,-1.331686,-29.705595,34.374534,171.0,326.0,187.0
463,VSTE3,-33.916422,-54.374358,52.341992,377.0,396.0,324.0
465,JFEN3,-54.423077,-92.500000,129.762323,417.0,415.0,419.0


In [110]:
def combinar_rankings(df, fatores, pesos=None):
    if pesos is None:
        pesos = {fator['nome']: 1 for fator in fatores}

    # Calcular ranking combinado
    df_resultado = df.copy()
    df_resultado.loc[:, 'Ranking_Combinado'] = 0.0

    for fator in fatores:
        nome = fator['nome']
        peso = pesos.get(nome, 1)
        df_resultado.loc[:, 'Ranking_Combinado'] += df_resultado[f'{nome}_Rank'].fillna(0) * peso

    # Ordenar pelo ranking combinado
    return df_resultado.sort_values('Ranking_Combinado')

In [112]:
acoes_ranks_combinados = combinar_rankings(acoes_rank_fatores, acoes_fatores, pesos={
    'Momentum_6m': 0.5,
    'Momentum_12m': 0.3,
    'Volatilidade': 0.2
})
acoes_ranks_combinados

,Ticker,Momentum_6m,Momentum_12m,Volatilidade,Momentum_6m_Rank,Momentum_12m_Rank,Volatilidade_Rank,Ranking_Combinado
263,FRAS3,31.609075,58.282830,21.161581,24.0,17.0,43.0,25.7
180,TGMA3,34.539837,55.300583,25.318405,21.0,20.0,86.0,33.7
184,GEPA4,16.677749,53.368286,22.289399,58.0,21.0,49.0,45.1
105,BBSE3,17.692076,27.781606,17.756093,56.0,49.0,15.0,45.7
81,AURA33,75.404372,198.598902,40.832390,9.0,4.0,233.0,52.3
...,...,...,...,...,...,...,...,...
383,AMOB3,NaN,NaN,77.650956,422.0,419.0,390.0,414.7
465,JFEN3,-54.423077,-92.500000,129.762323,417.0,415.0,419.0,416.8
246,ATED3,NaN,NaN,94.108463,422.0,419.0,401.0,416.9
460,OIBR3,-89.565217,-92.941177,117.000155,421.0,416.0,414.0,418.1


In [113]:
def factor_investing(estrategia='multifator', top_n=30, liquidez_minima=1_000_000):
    print(f"Aplicando estratégia: {estrategia}")

    # Obter dados fundamentalistas
    # print("Obtendo dados fundamentalistas...")
    # acoes_df = obter_dados_fundamentus()

    # Filtro de liquidez
    df_liquido = acoes_df[acoes_df['Liq.2meses'] > liquidez_minima].copy()

    # Definir fatores conforme a estratégia
    if estrategia == 'valor':
        fatores = [
            {'nome': 'PL', 'coluna': 'P/L', 'ascendente': True},
            {'nome': 'PVP', 'coluna': 'P/VP', 'ascendente': True},
            {'nome': 'EVEBIT', 'coluna': 'EV/EBIT', 'ascendente': True}
        ]
        pesos = {'PL': 1, 'PVP': 1, 'EVEBIT': 1}

    elif estrategia == 'qualidade':
        fatores = [
            {'nome': 'ROE', 'coluna': 'ROE', 'ascendente': False},
            {'nome': 'ROIC', 'coluna': 'ROIC', 'ascendente': False},
            {'nome': 'MargLiq', 'coluna': 'Mrg. Líq.', 'ascendente': False}
        ]
        pesos = {'ROE': 1, 'ROIC': 1, 'MargLiq': 1}

    elif estrategia == 'crescimento':
        fatores = [
            {'nome': 'CrescRec', 'coluna': 'Cresc. Rec.5a', 'ascendente': False}
        ]
        pesos = {'CrescRec': 1}

    elif estrategia == 'dividendos':
        fatores = [
            {'nome': 'DY', 'coluna': 'Div.Yield', 'ascendente': False}
        ]
        pesos = {'DY': 1}

    elif estrategia == 'momentum':
        print("Obtendo dados históricos para momentum...")
        tickers = df_liquido['Papel'].tolist()
        # dados_hist = obter_dados_historicos(tickers)

        # Calcular métricas técnicas
        df_tecnico = calcular_metricas_tecnicas(acoes_historico_dict)

        # Mesclar com dados fundamentalistas
        df_liquido = pd.merge(df_liquido, df_tecnico, left_on='Papel', right_on='Ticker', how='left')

        fatores = [
            {'nome': 'Mom6m', 'coluna': 'Momentum_6m', 'ascendente': False},
            {'nome': 'Mom12m', 'coluna': 'Momentum_12m', 'ascendente': False}
        ]
        pesos = {'Mom6m': 0.7, 'Mom12m': 0.3}

    elif estrategia == 'baixa_volatilidade':
        print("Obtendo dados históricos para volatilidade...")
        tickers = df_liquido['Papel'].tolist()
        # dados_hist = obter_dados_historicos(tickers)

        df_tecnico = calcular_metricas_tecnicas(acoes_historico_dict)
        df_liquido = pd.merge(df_liquido, df_tecnico, left_on='Papel', right_on='Ticker', how='left')

        fatores = [
            {'nome': 'Vol', 'coluna': 'Volatilidade', 'ascendente': True}
        ]
        pesos = {'Vol': 1}

    elif estrategia == 'multifator':
        print("Obtendo dados históricos...")
        tickers = df_liquido['Papel'].tolist()
        # dados_hist = obter_dados_historicos(tickers)

        df_tecnico = calcular_metricas_tecnicas(acoes_historico_dict)
        df_liquido = pd.merge(df_liquido, df_tecnico, left_on='Papel', right_on='Ticker', how='left')

        fatores = [
            {'nome': 'PL', 'coluna': 'P/L', 'ascendente': True},
            {'nome': 'PVP', 'coluna': 'P/VP', 'ascendente': True},
            {'nome': 'ROIC', 'coluna': 'ROIC', 'ascendente': False},
            {'nome': 'Mom6m', 'coluna': 'Momentum_6m', 'ascendente': False},
            {'nome': 'DY', 'coluna': 'Div.Yield', 'ascendente': False},
            {'nome': 'Vol', 'coluna': 'Volatilidade', 'ascendente': True}
        ]
        pesos = {'PL': 1, 'PVP': 1, 'ROIC': 1.5, 'Mom6m': 1, 'DY': 0.5, 'Vol': 0.5}

    else:
        print("Estratégia inválida. Use 'valor', 'qualidade', 'crescimento', 'dividendos', 'momentum', 'baixa_volatilidade' ou 'multifator'")
        return None

    # Calcular rankings para cada fator
    print("Calculando rankings...")
    df_ranks = ranking_fatores(df_liquido, fatores)

    # Combinar rankings com pesos
    df_resultado = combinar_rankings(df_ranks, fatores, pesos)

    # Selecionar as top N ações
    top_acoes = df_resultado.head(top_n)

    return top_acoes

In [117]:
acoes_top = factor_investing(estrategia='multifator', top_n=15, liquidez_minima=1_000_000)
acoes_top

Aplicando estratégia: multifator
Obtendo dados históricos...
Calculando rankings...


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Momentum_6m,Momentum_12m,Volatilidade,PL_Rank,PVP_Rank,ROIC_Rank,Mom6m_Rank,DY_Rank,Vol_Rank,Ranking_Combinado
33,BEEF3,6.97,-2.72,-8.84,0.124,0.00,0.100,0.54,1.79,-0.24,...,25.585577,14.449912,43.784118,34.0,1.0,83.0,17.0,170.0,143.0,333.0
78,VLID3,25.11,5.39,1.14,0.946,13.35,0.698,2.03,5.25,4.77,...,25.506471,55.700730,45.730556,79.0,121.0,26.0,18.0,16.0,150.0,340.0
64,ISAE4,22.17,4.18,0.74,1.834,10.65,0.337,3.12,3.23,-0.96,...,-2.760031,-4.301076,20.447443,65.0,61.0,75.0,96.0,27.0,16.0,356.0
4,AURA33,36.19,-52.77,5.69,2.440,2.19,1.175,9.20,6.59,-2.71,...,75.404372,198.598902,40.832390,5.0,213.0,10.0,3.0,145.0,133.0,375.0
81,LAVV3,9.54,5.44,1.25,1.203,10.56,0.539,0.89,5.58,2.03,...,27.316067,24.017574,28.157002,82.0,141.0,61.0,15.0,28.0,73.0,380.0
77,SAPR3,5.44,5.32,0.76,1.200,5.28,0.401,5.69,3.55,-1.29,...,4.672241,15.071730,26.861520,78.0,66.0,64.0,64.0,98.0,61.0,383.5
79,SAPR11,27.59,5.40,0.77,1.218,5.62,0.407,5.77,3.61,-1.31,...,2.003341,11.590112,24.510670,80.0,70.0,64.0,77.0,95.0,41.0,391.0
132,TGMA3,34.14,8.35,2.45,1.078,7.56,1.695,5.01,6.58,7.39,...,34.539837,55.300583,25.318405,133.0,190.0,5.0,10.0,61.0,50.0,396.0
80,SAPR4,5.53,5.41,0.77,1.220,5.71,0.407,5.78,3.61,-1.31,...,0.799486,10.269273,26.092463,81.0,70.0,64.0,80.0,92.0,57.0,401.5
63,CMIG4,10.11,4.07,1.06,0.727,10.26,0.484,-15.12,4.04,-1.44,...,-5.692150,14.852203,24.845885,64.0,106.0,56.0,110.0,32.0,44.0,402.0


In [118]:
acoes_top.to_csv('acoes_15_top_multifator.csv', index=False)